# Sandpiper usage

Sandpiper is a WPS server that resolves rules for climatological impacts. To get started, first instatiate the client. Here, the client will try to connect to a remote `sandpiper` instance using the `url` parameter.

In [1]:
from birdy import WPSClient
import json
import requests
import os
from wps_tools.testing import get_target_url
from pkg_resources import resource_filename

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "sandpiper":
    os.chdir('../')

In [2]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("sandpiper")
print(f"Using sandpiper on {url}")

Using sandpiper on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/sandpiper/wps


In [3]:
sandpiper = WPSClient(url)

The list of available processes can be diplayed using the `?` command (ex. `bird?`). Furthermore, help for individual processes can be diplayed using the same command (ex. `bird.process?`).

In [4]:
# NBVAL_IGNORE_OUTPUT
sandpiper?
sandpiper.resolve_rules?

Signature:
sandpiper.resolve_rules(
    csv_content,
    date_range='2080',
    region='bc',
    geoserver='https://docker-dev03.pcic.uvic.ca/geoserver/bc_regions/ows',
    connection_string='postgres://ce_meta_ro@db3.pcic.uvic.ca/ce_meta_12f290b63791',
    ensemble='p2a_rules',
    loglevel='INFO',
    thredds=True,
)
Docstring:
Resolve climatological impacts rules

Parameters
----------
csv_content : string
    Contents of the 'rules' CSV file
date_range : {'2020', '2050', '2080'}string
    30 year period for data
region : {'bc', 'alberni_clayoquot', 'boreal_plains', 'bulkley_nechako', 'capital', 'cariboo', 'central_coast', 'central_kootenay', 'central_okanagan', 'columbia_shuswap', ...}string
    Impacted region
geoserver : string
    Geoserver URL
connection_string : string
    Database connection string
ensemble : string
    Ensemble name filter for data files
thredds : boolean
    Data from thredds server. It is not recommended to change from the default (True)
loglevel : {'CRITI

Now that we know which process we wish to run, we can use the docstring to ensure we provide the appropriate parameters. Below we use a `credentials.json` file to provide the full connection string to the process.

In [5]:
try:
    with open('credentials.json') as f:
        data = json.load(f)
        connection_string = data['connection_string']
except Exception as e:
    print("You must run `make credentials` to create the required file")
    raise e

if connection_string == 'not_set':
    print("You must set the `connection_string` param in the `credentials.json` in order to run this demo")

else:
    with open(resource_filename('tests', 'data/rules_small.csv'), "r") as csv_file:
        output = sandpiper.resolve_rules(
            csv_content=csv_file.read(),
            connection_string=connection_string
    )

Access the output with json_to_dict() or auto_construct_outputs() from wps_tools.output_handling

In [6]:
# NBVAL_IGNORE_OUTPUT
from wps_tools.output_handling import json_to_dict, auto_construct_outputs
json_to_dict(output.get()[0])

{'rule_snow': True, 'rule_hybrid': True, 'rule_rain': True}

In [7]:
auto_construct_outputs(output.get())

[{'rule_snow': True, 'rule_hybrid': True, 'rule_rain': True}]

Once the process has completed we can extract the results and ensure it is what we expected.

In [6]:
# NBVAL_IGNORE_OUTPUT
rules = json.loads(requests.get(output.get()[0]).content.decode('utf-8'))
print(rules)

{'rule_snow': True, 'rule_hybrid': True, 'rule_rain': True}


In [7]:
expected_rules = {
    'rule_snow': True,
    'rule_hybrid': True,
    'rule_rain': True
}

assert rules == expected_rules